Задание 1. Базовое изучение
Изучить представленный набор данных на основе описания его столбцов и выбрать 8 столбцов для дальнейшего изучения (среди них должны быть как числовые, так и категориальные). Провести расчет базовых метрик для них, кратко описать результаты.

In [1]:
import pandas as pd
import numpy as np

In [19]:
horse = pd.read_csv('horse_data.csv', names = range(1, 29), na_values = '?')
horse.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 28 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   1       299 non-null    float64
 1   2       300 non-null    int64  
 2   3       300 non-null    int64  
 3   4       240 non-null    float64
 4   5       276 non-null    float64
 5   6       242 non-null    float64
 6   7       244 non-null    float64
 7   8       231 non-null    float64
 8   9       253 non-null    float64
 9   10      268 non-null    float64
 10  11      245 non-null    float64
 11  12      256 non-null    float64
 12  13      244 non-null    float64
 13  14      196 non-null    float64
 14  15      194 non-null    float64
 15  16      53 non-null     float64
 16  17      198 non-null    float64
 17  18      182 non-null    float64
 18  19      271 non-null    float64
 19  20      267 non-null    float64
 20  21      135 non-null    float64
 21  22      102 non-null    float64
 22  23

In [28]:
horse_i = horse.iloc[:,[0, 1, 3, 4, 5, 7, 10, 19]]
horse_i.columns = ['surgery', 'Age', 'rectal temperature', 'pulse', 'respiratory rate', 'peripheral puls', 'pain', 'total protein']
horse_i.describe()

,surgery,Age,rectal temperature,pulse,respiratory rate,peripheral puls,pain,total protein
count,299.000000,300.000000,240.000000,276.000000,242.000000,231.000000,245.00000,267.000000
mean,1.397993,1.640000,38.167917,71.913043,30.417355,2.017316,2.95102,24.456929
std,0.490305,2.173972,0.732289,28.630557,17.642231,1.042428,1.30794,27.475009
min,1.000000,1.000000,35.400000,30.000000,8.000000,1.000000,1.00000,3.300000
25%,1.000000,1.000000,37.800000,48.000000,18.500000,1.000000,2.00000,6.500000
50%,1.000000,1.000000,38.200000,64.000000,24.500000,2.000000,3.00000,7.500000
75%,2.000000,1.000000,38.500000,88.000000,36.000000,3.000000,4.00000,57.000000
max,2.000000,9.000000,40.800000,184.000000,96.000000,4.000000,5.00000,89.000000


Выводы по первому заданию:
    1. Age - при значениях 1 или 2 в результатах значения получились более 2, следовательно в данных ошибка
    2. surgery - болшей части лошадей делали операции
    3. rectal temperature - средняя температура лошадей выше нормы 37,8.
    4. pulse - пульс сильно завышен
    5. pespiratore rate - средняя частота дыхания сильно завышена (8-10), коррелирует с пульсом
    6. peripheral puls - периферический пульс увеличенный, что также коррелируется с пульсом и частой дыхания.
    7. pain - среднее значение указывает на умеренную боль, не ясно как влияет.
    8. total protein - среднее 24,4 при норме 6-7,5. Медиана 7,5. Явно во второй половине сильное завышение нормы.

Задание 2. Работа с выбросами
В выбранных числовых столбцах найти выбросы, выдвинуть гипотезы об их причинах и проинтерпретировать результаты. Принять и обосновать решение о дальнейшей работе с ними.

In [34]:
horse_num = horse_i.loc[:,['rectal temperature', 'respiratory rate', 'total protein']]

In [40]:
#rectal temperature
q1 = horse_num['rectal temperature'].quantile(0.25)
q3 = horse_num['rectal temperature'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
remove_outliers = horse_num[horse_num['rectal temperature'].between(lower_bound, upper_bound, inclusive=True)].sort_values('rectal temperature')
res = pd.concat([horse_num, remove_outliers]).drop_duplicates(keep=False)
res.loc[~pd.isnull(res['rectal temperature']), :]

,rectal temperature,respiratory rate,total protein
20,39.9,60.0,6.1
44,35.4,24.0,69.0
75,39.7,NaN,57.0
80,36.4,35.0,6.4
99,39.6,51.0,8.0
118,36.5,30.0,75.0
141,36.0,30.0,6.8
238,36.1,NaN,7.0
251,36.6,18.0,7.1
259,40.8,42.0,7.4


Вывод:
    При норме температуры 37,5 считаю текущие выбросы нормальными. Видно что при повышении температуры растет частота дыхания, считаю это нормальным. Оставляю как есть

In [41]:
#respiratory rate
q1 = horse_num['respiratory rate'].quantile(0.25)
q3 = horse_num['respiratory rate'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
remove_outliers = horse_num[horse_num['respiratory rate'].between(lower_bound, upper_bound, inclusive=True)].sort_values('respiratory rate')
res = pd.concat([horse_num, remove_outliers]).drop_duplicates(keep=False)
res.loc[~pd.isnull(res['respiratory rate']), :]

,rectal temperature,respiratory rate,total protein
3,39.1,84.0,7.2
39,39.2,96.0,NaN
41,39.0,72.0,8.5
82,38.1,80.0,5.7
84,37.8,80.0,4.5
103,38.0,68.0,5.3
106,38.3,96.0,6.1
120,39.4,66.0,6.0
125,38.0,68.0,7.6
208,37.8,80.0,89.0


Вывод:
    Выбросы по частоте дыхания сильно выбиваются из среднего значения и медианы. При повышении или пониении температуры явной зависимости нет, тем более температура при выбросах немного выше среднего значения. Значения по общему белку при выбросах также ведут себя хаотично. Решение - удалить выбросы.

In [42]:
#respiratory rate
q1 = horse_num['total protein'].quantile(0.25)
q3 = horse_num['total protein'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
remove_outliers = horse_num[horse_num['total protein'].between(lower_bound, upper_bound, inclusive=True)].sort_values('total protein')
res = pd.concat([horse_num, remove_outliers]).drop_duplicates(keep=False)
res.loc[~pd.isnull(res['total protein']), :]

,rectal temperature,respiratory rate,total protein


Вывод:
    Ура!!! Выбросов нет.

Задание 3. Работа с пропусками
Рассчитать количество пропусков для всех выбранных столбцов. Принять и обосновать решение о методе работы с пропусками по каждому столбцу, сформировать датафрейм, в котором пропуски будут отсутствовать.

In [44]:
horse_i.head()

,surgery,Age,rectal temperature,pulse,respiratory rate,peripheral puls,pain,total protein
0,2.0,1,38.5,66.0,28.0,3.0,5.0,8.4
1,1.0,1,39.2,88.0,20.0,NaN,3.0,85.0
2,2.0,1,38.3,40.0,24.0,1.0,3.0,6.7
3,1.0,9,39.1,164.0,84.0,1.0,2.0,7.2
4,2.0,1,37.3,104.0,35.0,NaN,NaN,7.4


In [46]:
for col in horse_i.columns:
    pct_missing = horse_i[col].isnull().mean()
    print(f'{col} - {pct_missing :.1%}')

surgery - 0.3%
Age - 0.0%
rectal temperature - 20.0%
pulse - 8.0%
respiratory rate - 19.3%
peripheral puls - 23.0%
pain - 18.3%
total protein - 11.0%


Пропуски есть везде, никакой столбец не удалить.
Так как в столбце age есть 9, но нет 2. То предположу что вместо 9 надо проставить 2.

In [49]:
horse_i['Age'] = horse_i.Age.apply(lambda age: 2 if age == 9 else age)

<ipython-input-49-a5670bab9fe1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  horse_i['Age'] = horse_i.Age.apply(lambda age: 2 if age == 9 else age)


In [51]:
horse_i.describe()

,surgery,Age,rectal temperature,pulse,respiratory rate,peripheral puls,pain,total protein
count,299.000000,300.000000,240.000000,276.000000,242.000000,231.000000,245.00000,267.000000
mean,1.397993,1.080000,38.167917,71.913043,30.417355,2.017316,2.95102,24.456929
std,0.490305,0.271746,0.732289,28.630557,17.642231,1.042428,1.30794,27.475009
min,1.000000,1.000000,35.400000,30.000000,8.000000,1.000000,1.00000,3.300000
25%,1.000000,1.000000,37.800000,48.000000,18.500000,1.000000,2.00000,6.500000
50%,1.000000,1.000000,38.200000,64.000000,24.500000,2.000000,3.00000,7.500000
75%,2.000000,1.000000,38.500000,88.000000,36.000000,3.000000,4.00000,57.000000
max,2.000000,2.000000,40.800000,184.000000,96.000000,4.000000,5.00000,89.000000


In [53]:
horse_i.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   surgery             299 non-null    float64
 1   Age                 300 non-null    int64  
 2   rectal temperature  240 non-null    float64
 3   pulse               276 non-null    float64
 4   respiratory rate    242 non-null    float64
 5   peripheral puls     231 non-null    float64
 6   pain                245 non-null    float64
 7   total protein       267 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 18.9 KB


Удалим строки где 6 из 8 значений нет. Если данных нет, то результат будет не понятным

In [55]:
horse_i = horse_i.dropna(thresh=6)
for col in horse_i.columns:
    pct_missing = horse_i[col].isnull().mean()
    print(f'{col} - {pct_missing :.1%}')

surgery - 0.4%
Age - 0.0%
rectal temperature - 12.8%
pulse - 3.0%
respiratory rate - 12.8%
peripheral puls - 17.0%
pain - 12.5%
total protein - 5.3%


In [57]:
horse_i.describe()

,surgery,Age,rectal temperature,pulse,respiratory rate,peripheral puls,pain,total protein
count,264.000000,265.000000,231.000000,257.000000,231.000000,220.000000,232.000000,251.000000
mean,1.405303,1.079245,38.170563,71.867704,30.445887,1.981818,2.883621,25.140637
std,0.491883,0.270632,0.730990,28.751053,17.361010,1.037927,1.278964,27.835115
min,1.000000,1.000000,35.400000,30.000000,8.000000,1.000000,1.000000,3.300000
25%,1.000000,1.000000,37.800000,48.000000,20.000000,1.000000,2.000000,6.550000
50%,1.000000,1.000000,38.200000,64.000000,26.000000,1.000000,3.000000,7.500000
75%,2.000000,1.000000,38.500000,88.000000,36.000000,3.000000,4.000000,58.000000
max,2.000000,2.000000,40.800000,184.000000,96.000000,4.000000,5.000000,89.000000


In [60]:
horse_i.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 265 entries, 0 to 299
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   surgery             264 non-null    float64
 1   Age                 265 non-null    int64  
 2   rectal temperature  231 non-null    float64
 3   pulse               257 non-null    float64
 4   respiratory rate    231 non-null    float64
 5   peripheral puls     220 non-null    float64
 6   pain                232 non-null    float64
 7   total protein       251 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 18.6 KB


In [61]:
horse_i.mode()

,surgery,Age,rectal temperature,pulse,respiratory rate,peripheral puls,pain,total protein
0,1.0,1,38.0,48.0,20.0,1.0,3.0,7.0


Так как по surgery всего один пропуск - заполню его модой (не исказитрезультат)

Так как мода температуры попадает в среднее значение, метиаду и попадает в диапазон 0.25 - 0.75, то заполню модой.

In [62]:
#surgery
horse_i['surgery'].fillna(horse_i['surgery'].mode()[0], inplace=True)

In [65]:
#rectal temperature
horse_i['rectal temperature'].fillna(horse_i['rectal temperature'].mode()[0], inplace=True)

In [66]:
horse_i = horse_i.dropna(thresh=6)
for col in horse_i.columns:
    pct_missing = horse_i[col].isnull().mean()
    print(f'{col} - {pct_missing :.1%}')

surgery - 0.0%
Age - 0.0%
rectal temperature - 0.0%
pulse - 3.0%
respiratory rate - 12.8%
peripheral puls - 17.0%
pain - 12.5%
total protein - 5.3%


In [68]:
horse_i.describe()

,surgery,Age,rectal temperature,pulse,respiratory rate,peripheral puls,pain,total protein
count,265.000000,265.000000,265.000000,257.000000,231.000000,220.000000,232.000000,251.000000
mean,1.403774,1.079245,38.148679,71.867704,30.445887,1.981818,2.883621,25.140637
std,0.491582,0.270632,0.684686,28.751053,17.361010,1.037927,1.278964,27.835115
min,1.000000,1.000000,35.400000,30.000000,8.000000,1.000000,1.000000,3.300000
25%,1.000000,1.000000,37.800000,48.000000,20.000000,1.000000,2.000000,6.550000
50%,1.000000,1.000000,38.000000,64.000000,26.000000,1.000000,3.000000,7.500000
75%,2.000000,1.000000,38.500000,88.000000,36.000000,3.000000,4.000000,58.000000
max,2.000000,2.000000,40.800000,184.000000,96.000000,4.000000,5.000000,89.000000


В учебных целях puplse заполню средним значением, так пропусков всего 5%, думаю не отразится на итоговом результате.

In [72]:
#pulse
horse_i.pulse.fillna(horse_i.pulse.mean(), inplace=True)

In [73]:
horse_i.describe()

,surgery,Age,rectal temperature,pulse,respiratory rate,peripheral puls,pain,total protein
count,265.000000,265.000000,265.000000,265.000000,231.000000,220.000000,232.000000,251.000000
mean,1.403774,1.079245,38.148679,71.867704,30.445887,1.981818,2.883621,25.140637
std,0.491582,0.270632,0.684686,28.312080,17.361010,1.037927,1.278964,27.835115
min,1.000000,1.000000,35.400000,30.000000,8.000000,1.000000,1.000000,3.300000
25%,1.000000,1.000000,37.800000,48.000000,20.000000,1.000000,2.000000,6.550000
50%,1.000000,1.000000,38.000000,64.000000,26.000000,1.000000,3.000000,7.500000
75%,2.000000,1.000000,38.500000,88.000000,36.000000,3.000000,4.000000,58.000000
max,2.000000,2.000000,40.800000,184.000000,96.000000,4.000000,5.000000,89.000000


Так как respiratory rate и peripheral puls сильно коррелировались по пульсу. То заполню пропуски средним значением с группировкой.

In [79]:
#respiratory rate
horse_i['respiratory rate'].fillna(horse_i.groupby('pulse')['respiratory rate'].transform('mean'), inplace=True)

In [80]:
horse_i.describe()

,surgery,Age,rectal temperature,pulse,respiratory rate,peripheral puls,pain,total protein
count,265.000000,265.000000,265.000000,265.000000,263.000000,220.000000,232.000000,251.000000
mean,1.403774,1.079245,38.148679,71.867704,30.313688,1.981818,2.883621,25.140637
std,0.491582,0.270632,0.684686,28.312080,16.552815,1.037927,1.278964,27.835115
min,1.000000,1.000000,35.400000,30.000000,8.000000,1.000000,1.000000,3.300000
25%,1.000000,1.000000,37.800000,48.000000,20.000000,1.000000,2.000000,6.550000
50%,1.000000,1.000000,38.000000,64.000000,27.000000,1.000000,3.000000,7.500000
75%,2.000000,1.000000,38.500000,88.000000,36.000000,3.000000,4.000000,58.000000
max,2.000000,2.000000,40.800000,184.000000,96.000000,4.000000,5.000000,89.000000


In [78]:
#Остались пропуски
horse_i.loc[pd.isnull(horse_i['respiratory rate']), :]

,surgery,Age,rectal temperature,pulse,respiratory rate,peripheral puls,pain,total protein
50,2.0,1,38.6,46.0,NaN,1.0,1.0,9.1
148,1.0,1,38.3,132.0,NaN,3.0,2.0,8.0


In [87]:
horse_i['respiratory rate'].fillna(horse_i.groupby(['pain', 'peripheral puls'])['respiratory rate'].transform('mean'), inplace=True)

In [109]:
horse_i.describe()

,surgery,Age,rectal temperature,pulse,respiratory rate,peripheral puls,pain,total protein
count,265.000000,265.000000,265.000000,265.000000,265.000000,265.000000,232.000000,251.000000
mean,1.403774,1.079245,38.148679,71.867704,30.285535,1.940721,2.883621,25.140637
std,0.491582,0.270632,0.684686,28.312080,16.499905,0.985556,1.278964,27.835115
min,1.000000,1.000000,35.400000,30.000000,8.000000,1.000000,1.000000,3.300000
25%,1.000000,1.000000,37.800000,48.000000,20.000000,1.000000,2.000000,6.550000
50%,1.000000,1.000000,38.000000,64.000000,27.000000,1.660256,3.000000,7.500000
75%,2.000000,1.000000,38.500000,88.000000,36.000000,3.000000,4.000000,58.000000
max,2.000000,2.000000,40.800000,184.000000,96.000000,4.000000,5.000000,89.000000


In [111]:
horse_i['peripheral puls'].fillna(horse_i.groupby(['Age', 'pulse'])['peripheral puls'].transform('mean'), inplace=True)

In [105]:
horse_i.describe()

,surgery,Age,rectal temperature,pulse,respiratory rate,peripheral puls,pain,total protein
count,265.000000,265.000000,265.000000,265.000000,265.000000,263.000000,232.000000,251.000000
mean,1.403774,1.079245,38.148679,71.867704,30.285535,1.940721,2.883621,25.140637
std,0.491582,0.270632,0.684686,28.312080,16.499905,0.989311,1.278964,27.835115
min,1.000000,1.000000,35.400000,30.000000,8.000000,1.000000,1.000000,3.300000
25%,1.000000,1.000000,37.800000,48.000000,20.000000,1.000000,2.000000,6.550000
50%,1.000000,1.000000,38.000000,64.000000,27.000000,1.660256,3.000000,7.500000
75%,2.000000,1.000000,38.500000,88.000000,36.000000,3.000000,4.000000,58.000000
max,2.000000,2.000000,40.800000,184.000000,96.000000,4.000000,5.000000,89.000000


In [107]:
#Остались пропуски - оставшиеся 2 значения заменю средним по столбцу
horse_i['peripheral puls'].fillna(horse_i['peripheral puls'].mean(), inplace=True)

In [108]:
horse_i.describe()

,surgery,Age,rectal temperature,pulse,respiratory rate,peripheral puls,pain,total protein
count,265.000000,265.000000,265.000000,265.000000,265.000000,265.000000,232.000000,251.000000
mean,1.403774,1.079245,38.148679,71.867704,30.285535,1.940721,2.883621,25.140637
std,0.491582,0.270632,0.684686,28.312080,16.499905,0.985556,1.278964,27.835115
min,1.000000,1.000000,35.400000,30.000000,8.000000,1.000000,1.000000,3.300000
25%,1.000000,1.000000,37.800000,48.000000,20.000000,1.000000,2.000000,6.550000
50%,1.000000,1.000000,38.000000,64.000000,27.000000,1.660256,3.000000,7.500000
75%,2.000000,1.000000,38.500000,88.000000,36.000000,3.000000,4.000000,58.000000
max,2.000000,2.000000,40.800000,184.000000,96.000000,4.000000,5.000000,89.000000


pain не понятно как влияет и какая зависимость, так как срзнач и медиана указывают на среднюю боль. То если я подставлю среднее значение. > немного изменилось std, думаю в пределах погрешности.

In [114]:
horse_i['pain'].fillna(horse_i['pain'].mean(), inplace=True)

In [115]:
horse_i.describe()

,surgery,Age,rectal temperature,pulse,respiratory rate,peripheral puls,pain,total protein
count,265.000000,265.000000,265.000000,265.000000,265.000000,265.000000,265.000000,251.000000
mean,1.403774,1.079245,38.148679,71.867704,30.285535,1.940721,2.883621,25.140637
std,0.491582,0.270632,0.684686,28.312080,16.499905,0.985556,1.196361,27.835115
min,1.000000,1.000000,35.400000,30.000000,8.000000,1.000000,1.000000,3.300000
25%,1.000000,1.000000,37.800000,48.000000,20.000000,1.000000,2.000000,6.550000
50%,1.000000,1.000000,38.000000,64.000000,27.000000,1.660256,3.000000,7.500000
75%,2.000000,1.000000,38.500000,88.000000,36.000000,3.000000,4.000000,58.000000
max,2.000000,2.000000,40.800000,184.000000,96.000000,4.000000,5.000000,89.000000


Не буду ломать голову, по столбцу total protein осталось всего 14 пропусков и явной зависимости не увидел, заполню медианой.


In [121]:
horse_i['total protein'].fillna(horse_i['total protein'].median(), inplace=True)

In [122]:
horse_i.describe()

,surgery,Age,rectal temperature,pulse,respiratory rate,peripheral puls,pain,total protein
count,265.000000,265.000000,265.000000,265.000000,265.000000,265.000000,265.000000,265.000000
mean,1.403774,1.079245,38.148679,71.867704,30.285535,1.940721,2.883621,24.208679
std,0.491582,0.270632,0.684686,28.312080,16.499905,0.985556,1.196361,27.374018
min,1.000000,1.000000,35.400000,30.000000,8.000000,1.000000,1.000000,3.300000
25%,1.000000,1.000000,37.800000,48.000000,20.000000,1.000000,2.000000,6.600000
50%,1.000000,1.000000,38.000000,64.000000,27.000000,1.660256,3.000000,7.500000
75%,2.000000,1.000000,38.500000,88.000000,36.000000,3.000000,4.000000,56.000000
max,2.000000,2.000000,40.800000,184.000000,96.000000,4.000000,5.000000,89.000000


In [123]:
horse_i.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 265 entries, 0 to 299
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   surgery             265 non-null    float64
 1   Age                 265 non-null    int64  
 2   rectal temperature  265 non-null    float64
 3   pulse               265 non-null    float64
 4   respiratory rate    265 non-null    float64
 5   peripheral puls     265 non-null    float64
 6   pain                265 non-null    float64
 7   total protein       265 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 18.6 KB
